In [34]:
import pandas as pd
import numpy as np

def shuffle(dataframe):
	
	msk = np.random.rand(len(dataframe)) < 0.8
	
	train = dataframe[msk].copy(deep=True)
	test = dataframe[~msk].copy(deep=True)
	
	train = (train - train.min())/(train.max() - train.min())
	test = (test - test.min())/(test.max() - test.min())
	'''
	test = dataframe.head(1)
	train = dataframe.tail(len(dataframe)-1)
	'''
	return train,test

def euclid(x1,x2,length):
	distance = 0
	for i in range(length):
		distance += np.square(x1[i] - x2[i])
	return np.sqrt(distance)

def knn(train,test,k):

	pridected_class = []
	length = train.shape[1]
	for i in range(len(test)):
	
		dict = {}
		for j in range(len(train)):
			dist = euclid(test.iloc[i],train.iloc[j],length)
			dict[j] = dist
		#a list of tuple (index,distance)
		sorted_dict = sorted(dict.items() , key=lambda x : x[1])
		
		neigh = []
		
		for neighbour in range(k):
			neigh.append(sorted_dict[neighbour][0])
			
		#predict class
		class1 = 0
		class0 = 0
		for x in range(len(neigh)):
			res = train.iloc[neigh[x]][-1]
			if res == 0:
				class0 += 1
			if res == 1:
				class1 += 1
				
		if class1 > class0:
			pridected_class.append(1)
		elif class0 >= class1:
			pridected_class.append(0)
			
			
	return pridected_class		

def accuracy(test,pridected_class):
	len = test.shape[0]
	correct = 0
	for i in range(len):
		if test.iloc[i][-1] == pridected_class[i]:
			correct += 1
	return (correct/len)

from sklearn.model_selection import KFold, StratifiedShuffleSplit

kf = KFold(n_splits=5)
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)

df = pd.read_csv('Andhra_dataset2.csv')

df_1 = df[df.reslt==1]
df_1.fillna(train.mean(),inplace=True)
df_0 = df[df.reslt==0]
df_0.fillna(train.mean(),inplace=True)
df = pd.concat([df_0, df_1])

df = df.sample(frac=1).reset_index(drop=True)
kf.get_n_splits(df)
df.drop(['education','history'],axis=1,inplace=True)
acclist = []
for train_index, test_index in kf.split(df):
    train, test = df.iloc[train_index], df.iloc[test_index]
    train.fillna(train.mean(),inplace=True)
    test.fillna(test.mean(),inplace=True)
    predicted_class = knn(train, test, 3)
    print(accuracy( test, predicted_class))
    acclist.append(accuracy( test, predicted_class))
print("total accuracy", sum(acclist)/len(acclist))

C:\Users\Pradyumna\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


0.9047619047619048
0.85
0.7
0.85
0.95
total accuracy 0.8509523809523809
